סיפריות והתקנות

In [7]:
pip install praw

In [8]:
import praw
import pandas as pd
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk

# להוריד את הנתונים הנדרשים לסנטימנט אנליזר אם הם לא קיימים
nltk.download('vader_lexicon')

# יצירת סנטימנט אנליזר
sia = SentimentIntensityAnalyzer()


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


אופציה 1 להורדת דאטה

הורדת דאטה באמצעות api
בעצם לקחנו את הapi של reddit והתחלנו לחלץ ממנו מידע כל דבר אפשרי שהיה ניתן לחלץ חילצנו
לקחנו את הפריטים החשובים וחקרנו איזה נושאים יכולים לעורר באזז (רגש/עניין חיובי או שלילי)

הנושאים שעלפיהם הורדנו נתונים והורדנו דאטה עשינו שינוי לפי הטיפוס top/hot:

*   **חדשות**: "news", "politics", "technology", "science", "environment"
*   **טרנדים ומגמות**: "AskReddit", "todayilearned", "interestingasfuck", "nextfuckinglevel"
*   **יוצרי תוכן**: "writingprompts", "creativity"
*   **פרסום ושיווק**: "advertising"
*   **נושאים שנויים במחלוקת**: "unpopularopinion", "changemyview", "conspiracy", "atheism", "religion", "relationships"
*   **תרבות פופולרית**: "meme", "movies", "gaming", "music", "television"

In [11]:
# 1. התחברות ל-Reddit API
reddit = praw.Reddit(
    client_id="TBV5poL7PvVKFF0sPo5PqQ",  # הכנס את ה-client_id שלך
    client_secret="1IfGXxHJSnd81HeVAxZaKS3pkhqjDw",  # הכנס את ה-client_secret שלך
    user_agent="Project"  # שם מותאם אישית לאפליקציה שלך
)

# 2. הגדרות לאיסוף הפוסטים
subreddit_name = "DebateReligion"
posts_limit = 100  # מספר הפוסטים לאיסוף

# רשימת אחסון נתונים
data = []

# 3. איסוף פוסטים
subreddit = reddit.subreddit(subreddit_name)
for post in subreddit.hot(limit=posts_limit):  # 'hot', אפשר לשנות ל-new/top
    title = post.title
    selftext = post.selftext
    created_at = post.created_utc
    upvote_ratio = post.upvote_ratio
    num_comments = post.num_comments
    subreddit_name = post.subreddit.display_name
    post_flair = post.link_flair_text if post.link_flair_text else "None"
    author_link_karma = getattr(post.author, "link_karma", None) if post.author else None
    account_age = getattr(post.author, "created_utc", None) if post.author else None

    upvotes = int(post.score * post.upvote_ratio)
    downvotes = post.score - upvotes
    media_type = "text" if post.is_self else ("link" if post.url else "media")
    media_details = post.url if not post.is_self else "None"
    title_length = len(title)
    post_length = len(selftext) if selftext else 0
    comments_to_upvotes_ratio = num_comments / upvotes if upvotes != 0 else 0
    author_name = post.author.name if post.author else None


##### NLP מתאים מאוד   ולחיזוי מעורבות או מחלוקת VADER ניתוח סנטימנט עם #####
    # ניתוח סנטימנט
    text_to_analyze = title + " " + selftext if selftext else title
    sentiment_scores = sia.polarity_scores(text_to_analyze)
    compound_score = sentiment_scores["compound"]
    sentiment_label = (
        "Positive" if compound_score > 0.05 else
        "Negative" if compound_score < -0.05 else
        "Neutral"
    )

###########################

    # הוספת המידע לרשימה
    data.append({
        "Title": title,
        "Title Length": title_length,
        "Selftext": selftext,
        "Post Length": post_length,
        "Subreddit": subreddit_name,
        "Upvotes": upvotes,
        "Downvotes": downvotes,
        "Number of Comments": num_comments,
        "Upvote Ratio": upvote_ratio,
        "Comments-to-Upvotes Ratio": comments_to_upvotes_ratio,
        "Created At": pd.to_datetime(created_at, unit='s').date(),
        "Post Flair": post_flair,
        "Author Name": author_name,
        "Author Link Karma": author_link_karma,
        "Account Age": pd.to_datetime(account_age, unit='s').date() if account_age else None,
        "Media Type": media_type,
        "Sentiment": sentiment_label  # עמודת סנטימנט
    })

# 4. שמירת הנתונים לקובץ CSV
df = pd.DataFrame(data)
df.to_csv("reddit_posts_features_with_sentiment.csv", index=False, encoding="utf-8")
print("הנתונים נשמרו בקובץ reddit_posts_features_with_sentiment.csv")

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/l

הנתונים נשמרו בקובץ reddit_posts_features_with_sentiment.csv


אופציה 2 להורדה

אופציה 2 להורדת דאטה

הורדת דאטה בלי הגבלת זמן וכמות גדולה יותר

בעצם הורדנו את הדאטה מטוחח רחב יותר עם עוד נושאים
 צורך זה נוצר שגילינו שהמודל שלנו לא מסווג מספיק ולא מגיע לדיוק שבעצם מרצה אותנו
לגכן בחרנו הגדלת הדאטה והרחבת המידעה הקיים בו ללא קשר להגבלת הapi של reddit
 שמביאה לנו מטווח קצר יותר ולא מורחב דבר שיכול להוביל אותנו למסקנות מוטעות


במהלך ההורדה שלנו גילינו שחלק מהנושאים לא מכילים כמות גדולה שניתן להוריד לכן הדאטה שלנו מוגבל מהבחינה הזאת  


In [13]:
# התחברות ל-Reddit
reddit = praw.Reddit(
    client_id="TBV5poL7PvVKFF0sPo5PqQ",
    client_secret="1IfGXxHJSnd81HeVAxZaKS3pkhqjDw",
    user_agent="Project"
)

# הגדרות
posts_limit = 600  # מספר פוסטים מכל סאב

# סאבים לפי קטגוריה
neutral_subreddits = ["aww", "dataisbeautiful", "DIY", "todayilearned", "LifeProTips", "technology", "food"]
controversial_subreddits = [
    "news", "politics", "science", "environment", "AskReddit",
    "interestingasfuck", "nextfuckinglevel", "writingprompts", "creativity",
    "advertising", "unpopularopinion", "changemyview", "conspiracy",
    "atheism", "religion", "relationships", "meme", "movies", "gaming", "music", "television"
]

all_data = []

# איסוף מכל סאב
for subreddit_name in neutral_subreddits + controversial_subreddits:
    category = "Neutral" if subreddit_name in neutral_subreddits else "Controversial"
    print(f"אוסף נתונים מ-{subreddit_name} ({category})...")

    try:
        subreddit = reddit.subreddit(subreddit_name)
        for post in subreddit.top(time_filter="all", limit=posts_limit):

            title = post.title
            selftext = post.selftext
            created_at = post.created_utc
            upvote_ratio = post.upvote_ratio
            num_comments = post.num_comments
            post_flair = post.link_flair_text if post.link_flair_text else "None"
            author_link_karma = post.author.link_karma if post.author else None
            account_age = post.author.created_utc if post.author else None
            upvotes = int(post.score * post.upvote_ratio)
            downvotes = post.score - upvotes
            media_type = "text" if post.is_self else ("link" if post.url else "media")
            title_length = len(title)
            post_length = len(selftext) if selftext else 0
            comments_to_upvotes_ratio = num_comments / upvotes if upvotes != 0 else 0
            author_name = post.author.name if post.author else None

            # סנטימנט
            text_to_analyze = title + " " + selftext if selftext else title
            sentiment_scores = sia.polarity_scores(text_to_analyze)
            compound_score = sentiment_scores["compound"]
            sentiment_label = (
                "Positive" if compound_score > 0.05 else
                "Negative" if compound_score < -0.05 else
                "Neutral"
            )

            all_data.append({
                "Title": title,
                "Title Length": title_length,
                "Selftext": selftext,
                "Post Length": post_length,
                "Subreddit": subreddit_name,
                "Upvotes": upvotes,
                "Downvotes": downvotes,
                "Number of Comments": num_comments,
                "Upvote Ratio": upvote_ratio,
                "Comments-to-Upvotes Ratio": comments_to_upvotes_ratio,
                "Created At": pd.to_datetime(created_at, unit='s').date(),
                "Post Flair": post_flair,
                "Author Name": author_name,
                "Author Link Karma": author_link_karma,
                "Account Age": pd.to_datetime(account_age, unit='s').date() if account_age else None,
                "Media Type": media_type,
                "Sentiment": sentiment_label,
                "Category": category
            })

        print(f"הסתיים איסוף מ-{subreddit_name}, ממתין 70 שניות...")
        time.sleep(70)

    except Exception as e:
        print(f"שגיאה בסאב {subreddit_name} - מדלג. הודעת שגיאה: {e}")
        continue

# המרה ל-DataFrame ושמירה
df = pd.DataFrame(all_data)
df.to_csv("reddit_combined_dataset.csv", index=False, encoding="utf-8")
print("✅ כל הנתונים נשמרו בקובץ reddit_combined_dataset.csv")


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



אוסף נתונים מ-aww (Neutral)...


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



שגיאה בסאב aww - מדלג. הודעת שגיאה: 'Redditor' object has no attribute 'link_karma'
אוסף נתונים מ-dataisbeautiful (Neutral)...


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/l

...

שגיאה בסאב writingprompts - מדלג. הודעת שגיאה: 'Redditor' object has no attribute 'link_karma'
אוסף נתונים מ-creativity (Controversial)...


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/l

שגיאה בסאב creativity - מדלג. הודעת שגיאה: 'Redditor' object has no attribute 'link_karma'
אוסף נתונים מ-advertising (Controversial)...


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



KeyboardInterrupt: 

אופציה 3 להורדת נתונים


ניסינו להוריד פוסטים מהחודש האחרון והגדלת מרחב הנושאים לדאטה שלנו

נושאים שהורדו מחדש
politics, technology, worldnews, AITA, relationship

r/AskReddit, r/todayilearned, r/explainlikeimfive, r/WritingPrompts

r/Conservative, r/LateStageCapitalism, r/UpliftingNews

r/programming, r/medicine, r/science

In [14]:

# התחברות ל-Reddit
reddit = praw.Reddit(
    client_id="TBV5poL7PvVKFF0sPo5PqQ",
    client_secret="1IfGXxHJSnd81HeVAxZaKS3pkhqjDw",
    user_agent="Project"
)

# הגדרות
posts_limit = 600  # מספר פוסטים
subreddit_name = "LegalAdvice"
category = "Controversial"
print(f"אוסף נתונים מ-{subreddit_name} ({category}) לחודש האחרון...")

all_data = []

try:
    subreddit = reddit.subreddit(subreddit_name)
    for post in subreddit.top(time_filter="month", limit=posts_limit):

        title = post.title
        selftext = post.selftext
        created_at = post.created_utc
        upvote_ratio = post.upvote_ratio
        num_comments = post.num_comments
        post_flair = post.link_flair_text if post.link_flair_text else "None"
        author_link_karma = post.author.link_karma if post.author else None
        account_age = post.author.created_utc if post.author else None
        upvotes = int(post.score * post.upvote_ratio)
        downvotes = post.score - upvotes
        media_type = "text" if post.is_self else ("link" if post.url else "media")
        title_length = len(title)
        post_length = len(selftext) if selftext else 0
        comments_to_upvotes_ratio = num_comments / upvotes if upvotes != 0 else 0
        author_name = post.author.name if post.author else None

        # סנטימנט
        text_to_analyze = title + " " + selftext if selftext else title
        sentiment_scores = sia.polarity_scores(text_to_analyze)
        compound_score = sentiment_scores["compound"]
        sentiment_label = (
            "Positive" if compound_score > 0.05 else
            "Negative" if compound_score < -0.05 else
            "Neutral"
        )

        all_data.append({
            "Title": title,
            "Title Length": title_length,
            "Selftext": selftext,
            "Post Length": post_length,
            "Subreddit": subreddit_name,
            "Upvotes": upvotes,
            "Downvotes": downvotes,
            "Number of Comments": num_comments,
            "Upvote Ratio": upvote_ratio,
            "Comments-to-Upvotes Ratio": comments_to_upvotes_ratio,
            "Created At": pd.to_datetime(created_at, unit='s').date(),
            "Post Flair": post_flair,
            "Author Name": author_name,
            "Author Link Karma": author_link_karma,
            "Account Age": pd.to_datetime(account_age, unit='s').date() if account_age else None,
            "Media Type": media_type,
            "Sentiment": sentiment_label,
            "Category": category
        })

    print(f"הסתיים איסוף מ-{subreddit_name}")

except Exception as e:
    print(f"שגיאה בסאב {subreddit_name} - הודעת שגיאה: {e}")

# המרה ל-DataFrame ושמירה
df = pd.DataFrame(all_data)
df.to_csv(f"reddit_{subreddit_name}_month_dataset.csv", index=False, encoding="utf-8")
print(f"✅ הנתונים נשמרו בקובץ reddit_{subreddit_name}_month_dataset.csv")


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



אוסף נתונים מ-LegalAdvice (Controversial) לחודש האחרון...


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/l

שגיאה בסאב LegalAdvice - הודעת שגיאה: 'Redditor' object has no attribute 'link_karma'
✅ הנתונים נשמרו בקובץ reddit_LegalAdvice_month_dataset.csv
